In [1]:
import math 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_q2_q3.xlsx to data_q2_q3.xlsx


In [3]:
#linear regression with the L2-norm regularization (Ridge regression)
#approach using BGD, SGD, and MBGD algorithms.

In [4]:
def cost_function(X,y,w,reg):        ###define·cost·function·
    hypothesis = np.dot(X,w.T)   ###calculation·of·hypothesis·for·all·instances···
    J = (1/(2*len(y))) * np.sum((hypothesis - y)**2) + (reg/2)*np.sum(np.square(w))
    return J

In [5]:
def batch_gradient_descent(X,y,w,alpha,reg,iters):
  
  cost_history = np.zeros(iters) # cost function for each iteration
  
  #initalize our cost history list to store the cost function on every iteration
  
  for i in range(iters):
    
    hypothesis = np.dot(X,w.T)
    w = w*(1-alpha*reg) - (alpha/len(y)) * np.dot((hypothesis-y), X)
    cost_history[i] = cost_function(X,y,w,reg)

  return w,cost_history

In [18]:
def stochastic_gradient_descent(X,y,w,alpha,reg, iters):
  
  cost_history = np.zeros(iters)
 
  for i in range(iters):
    
    rand_index = np.random.randint(len(y))
    ind_x = X[rand_index:rand_index+1]
    ind_y = y[rand_index:rand_index+1]
    w = w*(1-alpha*reg) - alpha * (ind_x.T.dot(ind_x.dot(w) - ind_y))
    cost = cost_function(ind_x,ind_y,w,reg)
  
  return w, cost

In [23]:
def MB_gradient_descent(X,y,w,alpha, iters, reg,batch_size):
  cost_history = np.zeros(iters)
  for i in range(iters):
    rand_index = np.random.randint(len(y))
    ind_x = X[rand_index:rand_index+batch_size]
    ind_y = y[rand_index:rand_index+batch_size]
    w = w*(1-alpha*reg) - (alpha/batch_size) * (ind_x.T.dot(ind_x.dot(w) - ind_y))
    cost = cost_function(ind_x,ind_y,w)
  
  return w, cost

In [8]:
# A function to implement min-max normalization

def norm(datan):
  data_min = np.min(datan, axis = 0)
  data_max = np.max(datan, axis = 0)
  datan = (datan- data_min)/(data_max-data_min)
  return datan


In [9]:
data = pd.read_excel(uploaded['data_q2_q3.xlsx'])

In [10]:
#test_train_validaion_split

data = norm(data) #all features of data are now normalized

df = data.sample(len(data)) #randomized data frame
df = df.values
train_len = int(0.7 * len(data))

test_len = int(0.2 * len(data))

X_train = df[0: train_len,[0,1,2,3]]                      #train
X_test = df[train_len: train_len+test_len,[0,1,2,3]]     #test
X_vald = df[train_len+test_len:,[0,1,2,3]]                 #validation
l = len(X_train)
m = len(X_test)
n = len(X_vald) 
X_train = np.append(np.ones([l, 1]),X_train, axis=1) #Column of ones
X_test = np.append(np.ones([m, 1]),X_test, axis=1) #Column of ones
X_vald = np.append(np.ones([n, 1]),X_vald, axis=1) #Column of ones

y_train = df[0: train_len,4]                    #train
y_test = df[train_len: train_len+test_len,4]     #test
y_vald = df[train_len+test_len:,4]               #validation

In [11]:
X_train.shape[1]

5

In [12]:
iters=500
#grid search
alpha_vals=np.linspace(0.0001,0.001,100)
reg_vals=np.linspace(0.1,1,10)
train_errors = []
W_values = []
i=0
for alpha in alpha_vals:
  for reg in reg_vals:
    w= np.zeros((5))
    batch_w,j_his = batch_gradient_descent(X_train,y_train,w,alpha,reg,iters)
    train_errors.append(j_his[iters -1])
    W_values.append(batch_w)
    i = i+1 

#best model
min_index = 0
for i in range(len(train_errors)):
  if(train_errors[i] < train_errors[min_index]):
    min_index = i

print(min_index)
print( 'optimal w values',W_values[min_index])

990
[0.22491164 0.1602437  0.10383832 0.1062071  0.09526949]


In [13]:
W_min = W_values[min_index]

In [14]:
#Mean square error
y_pred = np.dot(X_test,W_min.T)
Mean_square_error =  1/len(y_pred) * np.sum((y_pred - y_test)**2)     #Mean square error

Mean_abs_error =   1/len(y_pred) * np.sum(np.abs(y_pred - y_test))
corr_coff = np.corrcoef(y_pred,y_test) 
print("MSE for SGD with L2 = ",Mean_square_error)
print("MAE for SGD with L2= ",Mean_abs_error)
print("CC for SGD with L2 = ",corr_coff[0,1])

MSE for SGD with L2 =  0.17657093127011386
MAE for SGD with L2=  0.38809203453229507
CC for SGD with L2 =  0.728852950682363


In [27]:
iters=500
#grid search
alpha_vals=np.linspace(0.0001,0.001,100)
reg_vals=np.linspace(0.1,1,10)
train_errors = []
W_values = []
i=0
for alpha in alpha_vals:
  for reg in reg_vals:
    w= np.zeros((5))
    w_SGD,j_his = stochastic_gradient_descent(X_train,y_train,w,alpha,reg,iters)
    train_errors.append(j_his)
    W_values.append(w_SGD)
    i = i+1 

#best model
min_index = 0
for i in range(len(train_errors)):
  if(train_errors[i] < train_errors[min_index]):
    min_index = i


print('optimal w values',W_values[min_index])
W_min = W_values[min_index]
#Mean square error
y_pred = np.dot(X_test,W_min.T)
Mean_square_error =  1/len(y_pred) * np.sum((y_pred - y_test)**2)     #Mean square error

Mean_abs_error =   1/len(y_pred) * np.sum(np.abs(y_pred - y_test))
corr_coff = np.corrcoef(y_pred,y_test) 
print("MSE for SGD with L2 = ",Mean_square_error)
print("MAE for SGD with L2= ",Mean_abs_error)
print("CC for SGD with L2 = ",corr_coff[0,1])

optimal w values [0.04801007 0.0325622  0.02358152 0.02417115 0.01875058]
MSE for SGD with L2 =  0.4988711384427089
MAE for SGD with L2=  0.6103949903945742
CC for SGD with L2 =  0.7030664551957814


In [ ]:
iters=500
#grid search
alpha_vals=np.linspace(0.0001,0.001,100)
reg_vals=np.linspace(0.1,1,10)
train_errors = []
W_values = []
i=0
for alpha in alpha_vals:
  for reg in reg_vals:
    w= np.zeros((5))
    w_MBGD,j_his = MB_gradient_descent(X_train,y_train,w,alpha,reg,iters,30)
    train_errors.append(j_his[iters-1])
    W_values.append(w_SGD)
    i = i+1 

#best model
min_index = 0
for i in range(len(train_errors)):
  if(train_errors[i] < train_errors[min_index]):
    min_index = i


print('optimal w values',W_values[min_index])
W_min = W_values[min_index]
#Mean square error
y_pred = np.dot(X_test,W_min.T)
Mean_square_error =  1/len(y_pred) * np.sum((y_pred - y_test)**2)     #Mean square error

Mean_abs_error =   1/len(y_pred) * np.sum(np.abs(y_pred - y_test))
corr_coff = np.corrcoef(y_pred,y_test) 
print("MSE for MBGD with L2 = ",Mean_square_error)
print("MAE for MBGD with L2= ",Mean_abs_error)
print("CC for MBGD with L2 = ",corr_coff[0,1])